In [1]:
import pandas as pd
from seeq import spy
import livef1

In [2]:
# Constants

# Workbook name will scope all the signals to a particular workbook, Change to None if you want them 
# globally soped and available in any workbook 

WORKBOOK_NAME = None
DATASOURCE_NAME = 'F1 Telemetry Playground'
SESSION_CONDITION_NAME = 'F1 Sessions'
CURRENT_SEASON = 2025

pd.options.display.max_columns = None



In [3]:
def add_capsule_start_end_session_df (df):
    df['session_startDate'] = pd.to_datetime(df['Session Startdate'])
    df['gmtoffset'] = pd.to_timedelta(df['Gmtoffset'])
    df['Capsule Start'] = (df['Session Startdate'] + df['gmtoffset'])
    df['Capsule End'] = (df['Session Enddate'] + df['gmtoffset'])
    df['Capsule Start'] = pd.to_datetime(df['Capsule Start'], utc=True)
    df['Capsule End'] = pd.to_datetime(df['Capsule End'], utc=True)
    return df

def set_signals_max_interpolation():
    scoped_workbook_id = None
    if WORKBOOK_NAME != None:
        scoped_workbook = spy.search({'Name':WORKBOOK_NAME, 'Type':'Workbook'})
        scoped_workbook_id = scoped_workbook['ID'][0]
    
    telemetry_signals = spy.search({'Name':'20??.*.*.*','Type':'Signal','Scoped To':scoped_workbook_id})
    metadata = telemetry_signals.copy()
    metadata['Maximum Interpolation'] = '5 min'
    # Push the updated metadata back to Seeq
    spy.push(metadata=metadata)
    


def get_session_data(current_time):
    """Fetch session data from Seeq."""
    scoped_workbook_id = None
    if WORKBOOK_NAME != None:
        scoped_workbook = spy.search({'Name':WORKBOOK_NAME, 'Type':'Workbook'})
        scoped_workbook_id = scoped_workbook['ID'][0]
        
    posted_session_condition = spy.search({'Name': SESSION_CONDITION_NAME, 'Datasource Name': DATASOURCE_NAME, 'Scoped To':scoped_workbook_id})
    posted_sessions = spy.pull(posted_session_condition, start='2023-01-01T00:00:00Z', end=current_time)
    return posted_sessions[posted_sessions['Capsule End'] < current_time]

def filter_sessions(sessions_df, posted_sessions):
    """Filter sessions that have not been processed yet."""
    return sessions_df[~sessions_df['Session Key'].isin(posted_sessions['Session Key'])]

def process_telemetry_data(telemetry_data, current_season, current_team_name, current_driver_no):
    """Process telemetry data for a specific driver."""
    telemetry_data = telemetry_data.dropna(subset=['DriverNo'])
    telemetry_data['DriverNo'] = pd.to_numeric(telemetry_data['DriverNo'], errors='coerce').astype('Int64')
    telemetry_data = telemetry_data.drop(columns=['SessionKey', 'DriverNo', 'timestamp','TrackRegion','TrackStatus'])
    telemetry_data['Utc'] = pd.to_datetime(telemetry_data['Utc'])
    telemetry_data = telemetry_data.set_index('Utc')

    # Rename columns
    telemetry_data.columns = [
        f"{current_season}.{current_team_name}.{current_driver_no}.{col}" for col in telemetry_data.columns
    ]
    return telemetry_data

def process_laps_data(laps_data, current_team_name, current_driver_name):
    """Process laps data for a specific driver."""
    laps_data = laps_data.dropna(subset=['LapTime'])
    laps_data['team_name'] = current_team_name
    laps_data['current_driver_name'] = current_driver_name
    return laps_data

def create_sector_capsules(laps_data, sector_number):
    """Create sector capsules for a specific sector."""
    sector_caps = laps_data.copy()
    
    if sector_number == 1:
        sector_caps['Capsule Start'] = sector_caps['LapStartDate']
        sector_caps['Capsule End'] = sector_caps['Capsule Start'] + sector_caps['Sector1_Time']
    if sector_number == 2:
        sector_caps['Capsule Start'] = sector_caps['LapStartDate'] + sector_caps['Sector1_Time']
        sector_caps['Capsule End'] = sector_caps['Capsule Start'] + sector_caps['Sector2_Time']
    if sector_number == 3:
        sector_caps['Capsule Start'] = sector_caps['LapStartDate']+sector_caps['Sector1_Time']+sector_caps['Sector2_Time']
        sector_caps['Capsule End'] = sector_caps['Capsule Start'] + sector_caps['Sector3_Time']
    
    sector_caps['Sector'] = sector_number
    sector_caps = sector_caps[['LapNo', 'DriverNo', 'Capsule Start', 'Capsule End', 'Sector', 'team_name', 'current_driver_name']]
    return sector_caps.dropna(subset=['Capsule Start', 'Capsule End'])

def push_telemetry_data(telemetry_data):
    """Push telemetry data to Seeq."""
    spy.push(data=telemetry_data, workbook=WORKBOOK_NAME, datasource=DATASOURCE_NAME)

def push_sector_data(all_sectors, current_season, current_team_name, current_driver_no):
    """Push sector data to Seeq."""
    metadata = pd.DataFrame([{
        'Name': f"{current_season}.{current_team_name}.{current_driver_no}.Sectors",
        'Type': 'Condition',
        'Maximum Duration': '5 min',
    }])
    spy.push(data=all_sectors, metadata=metadata, workbook=WORKBOOK_NAME, datasource=DATASOURCE_NAME)

def process_session(session, session_key, car_number_info, current_season):
    """Process a single session."""
    laps_data = session.get_laps()
    laps_data = laps_data.dropna(subset = ['DriverNo'])
    laps_data['DriverNo'] = pd.to_numeric(laps_data['DriverNo'], errors='coerce').astype('Int64')
    
    telemetry_data = session.get_car_telemetry()
    telemetry_data = telemetry_data.dropna(subset = ['DriverNo'])
    telemetry_data['DriverNo'] = pd.to_numeric(telemetry_data['DriverNo'], errors='coerce').astype('Int64')

    cars_in_session = telemetry_data.DriverNo.unique()
    cars_pushed = 0

    for current_driver_no in cars_in_session:
        try:
            matching_row = car_number_info[
                (car_number_info['MinSessionKey'] <= session_key) &
                (car_number_info['MaxSessionKey'] >= session_key) &
                (car_number_info['DriverNo'] == current_driver_no)
            ]
    
            current_team_name = matching_row['TeamName'].values[0] if not matching_row.empty else None
            current_driver_name = matching_row['DriverName'].values[0] if not matching_row.empty else None
    
            # Process telemetry data
            driver_telemetry_data = process_telemetry_data(telemetry_data[telemetry_data.DriverNo == current_driver_no], current_season, current_team_name, current_driver_no)
            push_telemetry_data(driver_telemetry_data)
            print('Pushing Telemetry Data')
    
            # Process laps data
            driver_laps_data = process_laps_data(laps_data[laps_data.DriverNo == current_driver_no], current_team_name, current_driver_name)
            print('Pushing Lap Data')
    
            # Create sector capsules
            sector_1_caps = create_sector_capsules(driver_laps_data, 1)
            sector_2_caps = create_sector_capsules(driver_laps_data, 2)
            sector_3_caps = create_sector_capsules(driver_laps_data, 3)
            print('Creating Sectors')
    
            all_sectors = pd.concat([sector_1_caps, sector_2_caps, sector_3_caps], ignore_index=True)
            push_sector_data(all_sectors, current_season, current_team_name, current_driver_no)
            print('Pushing Sectors')
    
            cars_pushed += 1
        except Exception as e:
            print(f"Skipping car {current_driver_no} due to error: {e}")
        continue

    return cars_pushed



In [4]:
"""Main function to process sessions."""
current_time = pd.Timestamp.now(tz='UTC')

# Load season and car information
season = livef1.get_season(CURRENT_SEASON)
season.parse_sessions()
sessions_df = season.season_table
car_number_info = pd.read_csv("data/CarTeamMap.csv")

In [5]:
posted_sessions = get_session_data(current_time)
posted_sessions

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0F0A08D8-3F74-EA60-9A2B-CFBC2F46EC5F,StoredCondition,F1 Sessions,00:00:00.04,98,1,0 B,Success


,Condition,Capsule Start,Capsule End,Capsule Is Uncertain,Capsule ID,Meeting Key,Season Year,Meeting Offname,Meeting Circuit Shortname,Meeting Name,Session Type,Meeting Location,Meeting Country Name,Session Key,Session Name
0,F1 Sessions,2025-02-26 13:00:00+00:00,2025-02-26 22:00:00+00:00,False,235140596,1253,2025,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Sakhir,Pre-Season Testing,Practice 1,Sakhir,Bahrain,9683,Day 1
1,F1 Sessions,2025-02-27 13:00:00+00:00,2025-02-27 22:00:00+00:00,False,993166935,1253,2025,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Sakhir,Pre-Season Testing,Practice 2,Sakhir,Bahrain,9684,Day 2
2,F1 Sessions,2025-02-28 13:00:00+00:00,2025-02-28 22:00:00+00:00,False,-387375388,1253,2025,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Sakhir,Pre-Season Testing,Practice 3,Sakhir,Bahrain,9685,Day 3
3,F1 Sessions,2025-03-14 23:30:00+00:00,2025-03-15 00:30:00+00:00,False,-875040025,1254,2025,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,Melbourne,Australian Grand Prix,Practice 1,Melbourne,Australia,9686,Practice 1
4,F1 Sessions,2025-03-15 03:00:00+00:00,2025-03-15 04:00:00+00:00,False,-1845405065,1254,2025,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,Melbourne,Australian Grand Prix,Practice 2,Melbourne,Australia,9687,Practice 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,F1 Sessions,2025-10-17 07:30:00+00:00,2025-10-17 08:30:00+00:00,False,-591301615,1271,2025,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,Austin,United States Grand Prix,Practice 1,Austin,United States,9878,Practice 1
94,F1 Sessions,2025-10-17 11:30:00+00:00,2025-10-17 12:14:00+00:00,False,-1020042152,1271,2025,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,Austin,United States Grand Prix,Qualifying -1,Austin,United States,9879,Sprint Qualifying
95,F1 Sessions,2025-10-18 07:00:00+00:00,2025-10-18 08:00:00+00:00,False,-739851749,1271,2025,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,Austin,United States Grand Prix,Race -1,Austin,United States,9883,Sprint
96,F1 Sessions,2025-10-18 11:00:00+00:00,2025-10-18 12:00:00+00:00,False,-980020715,1271,2025,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,Austin,United States Grand Prix,Qualifying,Austin,United States,9884,Qualifying


In [6]:
sessions_df

,Season Year,Meeting Code,Meeting Number,Meeting Location,Meeting Offname,Meeting Name,Meeting Country Key,Meeting Country Code,Meeting Country Name,Meeting Circuit Key,Meeting Circuit Shortname,Session Key,Session Type,Session Name,Session Startdate,Session Enddate,Gmtoffset,Path
meeting_key,,,,,,,,,,,,,,,,,,
1253,2025,F12025T01,1,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Pre-Season Testing,36,BRN,Bahrain,63,Sakhir,9683,Practice 1,Day 1,2025-02-26 10:00:00,2025-02-26 19:00:00,03:00:00,2025/2025-02-28_Pre-Season_Testing/2025-02-26_...
1253,2025,F12025T01,1,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Pre-Season Testing,36,BRN,Bahrain,63,Sakhir,9684,Practice 2,Day 2,2025-02-27 10:00:00,2025-02-27 19:00:00,03:00:00,2025/2025-02-28_Pre-Season_Testing/2025-02-27_...
1253,2025,F12025T01,1,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Pre-Season Testing,36,BRN,Bahrain,63,Sakhir,9685,Practice 3,Day 3,2025-02-28 10:00:00,2025-02-28 19:00:00,03:00:00,2025/2025-02-28_Pre-Season_Testing/2025-02-28_...
1254,2025,F1202501,1,Melbourne,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,Australian Grand Prix,5,AUS,Australia,10,Melbourne,9686,Practice 1,Practice 1,2025-03-14 12:30:00,2025-03-14 13:30:00,11:00:00,2025/2025-03-16_Australian_Grand_Prix/2025-03-...
1254,2025,F1202501,1,Melbourne,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,Australian Grand Prix,5,AUS,Australia,10,Melbourne,9687,Practice 2,Practice 2,2025-03-14 16:00:00,2025-03-14 17:00:00,11:00:00,2025/2025-03-16_Australian_Grand_Prix/2025-03-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,2025,F1202519,19,Austin,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,United States Grand Prix,19,USA,United States,9,Austin,9878,Practice 1,Practice 1,2025-10-17 12:30:00,2025-10-17 13:30:00,-05:00:00,2025/2025-10-19_United_States_Grand_Prix/2025-...
1271,2025,F1202519,19,Austin,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,United States Grand Prix,19,USA,United States,9,Austin,9879,Qualifying -1,Sprint Qualifying,2025-10-17 16:30:00,2025-10-17 17:14:00,-05:00:00,2025/2025-10-19_United_States_Grand_Prix/2025-...
1271,2025,F1202519,19,Austin,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,United States Grand Prix,19,USA,United States,9,Austin,9883,Race -1,Sprint,2025-10-18 12:00:00,2025-10-18 13:00:00,-05:00:00,2025/2025-10-19_United_States_Grand_Prix/2025-...


In [7]:
try:
    posted_sessions = get_session_data(current_time)
    filtered_sessions = filter_sessions(sessions_df, posted_sessions)

except Exception as e:
    print(f"No prior sessions have been uploaded posting all sessions - error text: {e}")
    filtered_sessions = sessions_df
    posted_sessions = None

filtered_sessions = add_capsule_start_end_session_df(filtered_sessions)
 

,ID,Type,Name,Time,Count,Pages,Data Processed,Result
0,0F0A08D8-3F74-EA60-9A2B-CFBC2F46EC5F,StoredCondition,F1 Sessions,00:00:00.03,98,1,6 KB,Success


In [8]:
posted_sessions

,Condition,Capsule Start,Capsule End,Capsule Is Uncertain,Capsule ID,Meeting Key,Season Year,Meeting Offname,Meeting Circuit Shortname,Meeting Name,Session Type,Meeting Location,Meeting Country Name,Session Key,Session Name
0,F1 Sessions,2025-02-26 13:00:00+00:00,2025-02-26 22:00:00+00:00,False,235140596,1253,2025,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Sakhir,Pre-Season Testing,Practice 1,Sakhir,Bahrain,9683,Day 1
1,F1 Sessions,2025-02-27 13:00:00+00:00,2025-02-27 22:00:00+00:00,False,993166935,1253,2025,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Sakhir,Pre-Season Testing,Practice 2,Sakhir,Bahrain,9684,Day 2
2,F1 Sessions,2025-02-28 13:00:00+00:00,2025-02-28 22:00:00+00:00,False,-387375388,1253,2025,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,Sakhir,Pre-Season Testing,Practice 3,Sakhir,Bahrain,9685,Day 3
3,F1 Sessions,2025-03-14 23:30:00+00:00,2025-03-15 00:30:00+00:00,False,-875040025,1254,2025,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,Melbourne,Australian Grand Prix,Practice 1,Melbourne,Australia,9686,Practice 1
4,F1 Sessions,2025-03-15 03:00:00+00:00,2025-03-15 04:00:00+00:00,False,-1845405065,1254,2025,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,Melbourne,Australian Grand Prix,Practice 2,Melbourne,Australia,9687,Practice 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,F1 Sessions,2025-10-17 07:30:00+00:00,2025-10-17 08:30:00+00:00,False,-591301615,1271,2025,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,Austin,United States Grand Prix,Practice 1,Austin,United States,9878,Practice 1
94,F1 Sessions,2025-10-17 11:30:00+00:00,2025-10-17 12:14:00+00:00,False,-1020042152,1271,2025,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,Austin,United States Grand Prix,Qualifying -1,Austin,United States,9879,Sprint Qualifying
95,F1 Sessions,2025-10-18 07:00:00+00:00,2025-10-18 08:00:00+00:00,False,-739851749,1271,2025,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,Austin,United States Grand Prix,Race -1,Austin,United States,9883,Sprint
96,F1 Sessions,2025-10-18 11:00:00+00:00,2025-10-18 12:00:00+00:00,False,-980020715,1271,2025,FORMULA 1 MSC CRUISES UNITED STATES GRAND PRIX...,Austin,United States Grand Prix,Qualifying,Austin,United States,9884,Qualifying


In [9]:
filtered_sessions

,Season Year,Meeting Code,Meeting Number,Meeting Location,Meeting Offname,Meeting Name,Meeting Country Key,Meeting Country Code,Meeting Country Name,Meeting Circuit Key,Meeting Circuit Shortname,Session Key,Session Type,Session Name,Session Startdate,Session Enddate,Gmtoffset,Path,session_startDate,gmtoffset,Capsule Start,Capsule End
meeting_key,,,,,,,,,,,,,,,,,,,,,,


In [10]:
for index, row in filtered_sessions.iterrows():
    current_session_key = row['Session Key']
    session = livef1.get_session(season=CURRENT_SEASON, meeting_key=index, session_key=current_session_key)
    session_generated = False
    try:
        session.generate(silver=True)
        session_generated = True
    except Exception as e:
        print(f"Skipping session {row['session_key']} due to error: {e}")
        continue
        
    if session_generated:
        
        cars_pushed = process_session(session,current_session_key,car_number_info, CURRENT_SEASON)

        if cars_pushed > 0:
            # Push session metadata

            keep_cols = [
                "Capsule Start", 
                "Capsule End",
                "index", 
                "Season Year",
                "Meeting Location",
                "Meeting Offname",
                "Meeting Name",
                "Meeting Country Name",
                "Meeting Circuit Shortname",
                "Session Key",
                "Session Type",
                "Session Name"]
            
            session_capsule = pd.DataFrame([row]).reset_index()
            session_capsule = session_capsule[keep_cols]
            session_capsule = session_capsule.rename(columns={'index': 'Meeting Key'})
            
            spy.push(data=session_capsule, metadata=pd.DataFrame([{
                'Name': SESSION_CONDITION_NAME,
                'Type': 'Condition',
                'Maximum Duration': '7d',
            }]), workbook=WORKBOOK_NAME, datasource=DATASOURCE_NAME)


    

In [11]:
spy.jobs.schedule('Every 1 Day')

,Schedule,Scheduled,Next Run
0,Every 1 Day,At 12:00 AM,2025-10-24 00:00:00 PDT


,Schedule,Scheduled,Next Run
0,Every 1 Day,At 12:00 AM,2025-10-24 00:00:00 PDT
